In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
# Read the data
train_df = pd.read_csv('dataset/train_split_partially_preprocessed.csv')
test_df = pd.read_csv('dataset/test_split_partially_preprocessed.csv')
print("Train: ", train_df.shape, "\tTest: ", test_df.shape)

In [ ]:
# test_df = pd.read_csv('dataset/test_split_partially_preprocessed.csv')

Create state fields

In [ ]:
# num_nostate = (df.PWSTATE2 == 0).values.sum()
# original_len = len(df)
# print(f"{num_nostate} ({num_nostate/original_len:.2%}) of the rows have no state")

In [ ]:
# # Remove rows with no state
# df = df[df.PWSTATE2 != 0]
# assert(len(df) == original_len - num_nostate)
# print(f"Removed {num_nostate} rows with no state. {len(df)} rows remain.")

In [ ]:
# # Drop columns that are now unnecessary

# origNumCols = len(df.columns)
# droppedCols = []

# for col in df.columns:
#     unique = df[col].unique()
#     if(len(unique) == 1):
#         print(f"Dropping column {col} since it has only one value: {unique[0]}")
#         droppedCols.append(col)
#     elif(len(unique) == 2 and df[col].isna().values.any()):
#         print(f"Warning: Column {col} has two values but you may still want to drop it: {unique[0]} and {unique[1]}")

# df.drop(droppedCols, axis=1, inplace=True)
# assert(len(df.columns) == origNumCols - len(droppedCols))
# print(f"Dropped {len(droppedCols)} columns. {len(df.columns)} columns remain.")

In [ ]:
with(open('ipums_fields/stateField.json')) as f:
    state_mapping = json.load(f)

In [ ]:
# Make sure each degree field is in the mapping
stateKeys = sorted([int(k) for k in state_mapping.keys() if int(k) != 0])
# Note that stateKeys doesn't include foreign countries
assert(sorted([val for val in train_df.PWSTATE2.unique().tolist() if val <= stateKeys[-1]]) == stateKeys)
assert(sorted([val for val in test_df.PWSTATE2.unique().tolist() if val <= stateKeys[-1]]) == stateKeys)

In [ ]:
train_df["worksOutsideUS"] = (train_df.PWSTATE2 > stateKeys[-1])
test_df["worksOutsideUS"] = (test_df.PWSTATE2 > stateKeys[-1])

cols_created = 1
for (key, value) in state_mapping.items():
    # Skip N/A column (may want to fill this with NaN later)
    if value == 'N/A':
        continue

    stateName = value.replace(' ', '_')
    train_df[f"worksIn_{stateName}"] = (train_df.PWSTATE2 == int(key))
    test_df[f"worksIn_{stateName}"] = (test_df.PWSTATE2 == int(key))
    cols_created += 1

print(f"Created {cols_created} columns")

In [ ]:
train_df.drop(columns=['PWSTATE2', 'PWCOUNTY', 'PWTYPE'], inplace=True)
test_df.drop(columns=['PWSTATE2', 'PWCOUNTY', 'PWTYPE'], inplace=True)

Create degree fields

In [ ]:
with(open('ipums_fields/degField.json')) as f:
    deg_mapping = json.load(f)

In [ ]:
# Make sure each degree field is in the mapping
assert(sorted(train_df.DEGFIELD.unique().tolist()) == sorted([int(k) for k in deg_mapping.keys()]))
assert(sorted(test_df.DEGFIELD.unique().tolist()) == sorted([int(k) for k in deg_mapping.keys()]))

In [ ]:
cols_created = 0
for (key, value) in deg_mapping.items():
    # Skip N/A column (may want to fill this with NaN later)
    if value == 'N/A':
        continue

    degName = value.replace(' ', '_').replace(',', '')
    train_df[f"hasDegree_{degName}"] = (train_df.DEGFIELD == int(key))
    test_df[f"hasDegree_{degName}"] = (test_df.DEGFIELD == int(key))
    cols_created += 1

print(f"Created {cols_created} columns")

In [ ]:
train_df.drop(columns=['DEGFIELD', 'DEGFIELDD'], inplace=True)
test_df.drop(columns=['DEGFIELD', 'DEGFIELDD'], inplace=True)

Create occupation fields

In [ ]:
with(open('ipums_fields/occupation2010.json')) as f:
    occ_mapping = json.load(f)

In [ ]:
assert(sorted(train_df.OCC2010.unique().tolist()) == sorted(test_df.OCC2010.unique().tolist()))

In [ ]:
# Make sure each degree field is in the mapping
sortedOcc = sorted([int(k) for k in occ_mapping.keys()])
uniqueCodes = train_df.OCC2010.unique().tolist()
removedKeys = []

for jobCode in uniqueCodes:
    assert(jobCode in sortedOcc)

for jobCode in sortedOcc:
    if jobCode not in uniqueCodes:
        removedKeys.append(jobCode)
        print(f"No examples of job code {jobCode} (occupation {occ_mapping[str(jobCode)]})")
# assert(sorted(df.OCC2010.unique().tolist()) == )

In [ ]:
cols_created = 0
for (key, value) in occ_mapping.items():
    # Skip N/A column (may want to fill this with NaN later)
    if value == 'N/A':
        continue
    
    # Skip occupations that were removed
    if key in removedKeys:
        continue

    occName = value.replace(' ', '_').replace(',', '')
    train_df[f"occupation_{occName}"] = (train_df.OCC2010 == int(key))
    test_df[f"occupation_{occName}"] = (test_df.OCC2010 == int(key))
    cols_created += 1

print(f"Created {cols_created} columns")

In [ ]:
print(f"Created {cols_created} columns")

In [ ]:
train_df.drop(columns=['OCC2010'], inplace=True)
test_df.drop(columns=['OCC2010'], inplace=True)

In [ ]:
train_df = train_df.copy()
test_df = test_df.copy()

## Save dataframes

If you're going to do salary adjustments, don't save dataframes here.

In [ ]:
train_df.reset_index().to_csv('dataset/train_preprocessed.csv', index=False)

In [ ]:
test_df.reset_index().to_csv('dataset/test_preprocessed.csv', index=False)

## Optional Salary adjustments

The following cells contain optional adjustments to salary. Make sure not to run all of them since some are meant to be run alone.

#### Discrete adjustments

This creates 11 discrete buckets for classification, with each bucket containing a range of salaries of $10k

In [ ]:
for i in range(10):
    col_name = f"makes{i * 10}To{(i+1) * 10}K"
    train_df[col_name] = (train_df.INCWAGE_CPIU_2010 >= i * 10000) & (train_df.INCWAGE_CPIU_2010 < (i+1) * 10000)
    test_df[col_name] = (test_df.INCWAGE_CPIU_2010 >= i * 10000) & (test_df.INCWAGE_CPIU_2010 < (i+1) * 10000)

col_name = f"makesOver100K"
train_df[col_name] = (train_df.INCWAGE_CPIU_2010 >= 100000)
test_df[col_name] = (test_df.INCWAGE_CPIU_2010 >= 100000)

print("Cols created:", *[col for col in train_df.columns if col.startswith("makes") and train_df[col].dtype == bool and test_df[col].dtype == bool])

# train_df.drop(columns=['INCWAGE_CPIU_2010'], inplace=True)

In [ ]:
train_df.drop(columns=['INCWAGE_CPIU_2010'], inplace=True)
test_df.drop(columns=['INCWAGE_CPIU_2010'], inplace=True)

In [ ]:
train_df.reset_index().to_csv('dataset/train_preprocessed_buckets.csv', index=False)

In [ ]:
test_df.reset_index().to_csv('dataset/test_preprocessed_buckets.csv', index=False)

This also creates 11 discrete buckets for classification. Here, the buckets are created to hold the same number of people. Therefore, some buckets may contain a wider range of values than others.

If you run this part, make sure you haven't run the previous 4 cells which create buckets using a different method.

In [ ]:
# Conversion to uint not necessary but makes the file names more readable
train_df['INCWAGE_CPIU_2010'] = train_df['INCWAGE_CPIU_2010'].astype('uint')
test_df['INCWAGE_CPIU_2010'] = test_df['INCWAGE_CPIU_2010'].astype('uint')

In [ ]:
full_df = pd.concat([train_df, test_df], ignore_index=True)
assert(len(full_df) == len(train_df) + len(test_df))
full_df.sort_values(by=['INCWAGE_CPIU_2010'], inplace=True)
full_df.head()

In [ ]:
full_df = full_df.reset_index().drop(['index'], axis=1)
full_df.head()

In [ ]:
# This is just to make sure my indexing is correct
assert((full_df.INCWAGE_CPIU_2010.iloc[len(full_df)-1] == full_df.INCWAGE_CPIU_2010.max()) and (full_df.INCWAGE_CPIU_2010.iloc[0] == full_df.INCWAGE_CPIU_2010.min()))

In [ ]:
numBuckets = 11
elementsPerBucket = len(full_df) // numBuckets

cutoffs = []

# Get cutoffs for each bucket
for i in range(1, numBuckets):
    cutoffs.append(full_df.INCWAGE_CPIU_2010.iloc[i * elementsPerBucket])

cutoffs

In [ ]:
train_df['incomeBucket0'] = (train_df.INCWAGE_CPIU_2010 < cutoffs[0])
test_df['incomeBucket0'] = (test_df.INCWAGE_CPIU_2010 < cutoffs[0])

for i in range(1, numBuckets - 1):
    train_df[f'incomeBucket{i}'] = (train_df.INCWAGE_CPIU_2010 >= cutoffs[i-1]) & (train_df.INCWAGE_CPIU_2010 < cutoffs[i])
    test_df[f'incomeBucket{i}'] = (test_df.INCWAGE_CPIU_2010 >= cutoffs[i-1]) & (test_df.INCWAGE_CPIU_2010 < cutoffs[i])

train_df[f'incomeBucket{numBuckets - 1}'] = (train_df.INCWAGE_CPIU_2010 >= cutoffs[-1])
test_df[f'incomeBucket{numBuckets - 1}'] = (test_df.INCWAGE_CPIU_2010 >= cutoffs[-1])

In [ ]:
for df in [train_df, test_df]:
    # Check that every element is in at least 1 bucket
    vals = df.incomeBucket0
    for i in range(1, numBuckets):
        vals = vals | df[f'incomeBucket{i}']
    assert(vals.all())

    df.drop(columns=['INCWAGE_CPIU_2010'], inplace=True)

In [ ]:
train_df.reset_index().to_csv(f'dataset/train_variableBuckets_cutoffs{",".join([str(c) for c in cutoffs])}.csv', index=False)

In [ ]:
test_df.reset_index().to_csv(f'dataset/test_variableBuckets_cutoffs{",".join([str(c) for c in cutoffs])}.csv', index=False)

#### Continuous adjustments
These create continuous data for regression

Divide salaries by 5000 to reduce range of data

In [ ]:
train_df['INCWAGE_CPIU_2010'] = (train_df['INCWAGE_CPIU_2010'].astype(float) / 5000)
test_df['INCWAGE_CPIU_2010'] = (test_df['INCWAGE_CPIU_2010'].astype(float) / 5000)

Cap max salary at 100000

In [ ]:
# Set ceiling to $100k salary (assumes salaries divided by 5000)
train_df.loc[(train_df.INCWAGE_CPIU_2010 > 20), 'INCWAGE_CPIU_2010'] = 20
test_df.loc[(test_df.INCWAGE_CPIU_2010 > 20), 'INCWAGE_CPIU_2010'] = 20

# Both train and test should have at least 1 example of salaries >= $100k, in which case this assertion should pass
assert(train_df.INCWAGE_CPIU_2010.max() == 20 and test_df.INCWAGE_CPIU_2010.max() == 20)

Adjust range to [-10, 10] (assumes salary capped at 20)

In [ ]:
train_df['INCWAGE_CPIU_2010'] -= 10
test_df['INCWAGE_CPIU_2010'] -= 10

Adjust salaries to have mean at 0 (ranges will vary depending on data)

In [ ]:
# Note: mean of train/test should be approximately the same, so could remove the append here
meanAdjustment = pd.concat([train_df['INCWAGE_CPIU_2010'], test_df['INCWAGE_CPIU_2010']]).mean()
print(f"Mean is {meanAdjustment}, adjusting train/test data accordingly")

train_df['INCWAGE_CPIU_2010'] -= meanAdjustment
test_df['INCWAGE_CPIU_2010'] -= meanAdjustment

print(f"For train data: min is {train_df.INCWAGE_CPIU_2010.min()}, max is {train_df.INCWAGE_CPIU_2010.max()}")
print(f"For test data: min is {test_df.INCWAGE_CPIU_2010.min()}, max is {test_df.INCWAGE_CPIU_2010.max()}")

Adjust to normal distribution with mean=0 and std=1. Cutoffs specified below to avoid major outliers. Note that this assumes that only mean adjustment (the cell immediately above this one) was run.

In [ ]:
# These are the max/min values that will be allowed. Make sure they aren't too small/large since that may include/exclude too many values
max_cutoff = 5
min_cutoff = -5

# Note: std of train/test should be approximately the same, so could remove the append here
print(f"STD of train data is {train_df.INCWAGE_CPIU_2010.std()}. STD of test data is {test_df.INCWAGE_CPIU_2010.std()}. Ensure these values are similar.")
stdAdjustment = pd.concat([train_df['INCWAGE_CPIU_2010'], test_df['INCWAGE_CPIU_2010']]).std()
print(f"STD is {stdAdjustment}, adjusting train/test data accordingly")

train_df['INCWAGE_CPIU_2010'] /= stdAdjustment
test_df['INCWAGE_CPIU_2010'] /= stdAdjustment

print(f"For train data: min is {train_df.INCWAGE_CPIU_2010.min()}, max is {train_df.INCWAGE_CPIU_2010.max()}")
print(f"For test data: min is {test_df.INCWAGE_CPIU_2010.min()}, max is {test_df.INCWAGE_CPIU_2010.max()}")

train_df.loc[(train_df.INCWAGE_CPIU_2010 > max_cutoff), 'INCWAGE_CPIU_2010'] = max_cutoff
test_df.loc[(test_df.INCWAGE_CPIU_2010 > max_cutoff), 'INCWAGE_CPIU_2010'] = max_cutoff

train_df.loc[(train_df.INCWAGE_CPIU_2010 < min_cutoff), 'INCWAGE_CPIU_2010'] = min_cutoff
test_df.loc[(test_df.INCWAGE_CPIU_2010 < min_cutoff), 'INCWAGE_CPIU_2010'] = min_cutoff

print(f"For train data: adjusted min is {train_df.INCWAGE_CPIU_2010.min()}, adjusted max is {train_df.INCWAGE_CPIU_2010.max()}")
print(f"For test data: adjusted min is {test_df.INCWAGE_CPIU_2010.min()}, adjusted max is {test_df.INCWAGE_CPIU_2010.max()}")

In [ ]:
print(f'dataset/train_preprocessed_divby5k_meanAdjustedBy{meanAdjustment}_stdAdjustedBy{stdAdjustment}.csv')

In [ ]:
train_df.reset_index().to_csv(f'dataset/train_preprocessed_div5k_cap20_meanAdj{meanAdjustment}_stdAdj{stdAdjustment}_max{train_df.INCWAGE_CPIU_2010.max()}_min{train_df.INCWAGE_CPIU_2010.min()}.csv', index=False)

In [ ]:
train_df.shape

In [ ]:
test_df.reset_index().to_csv(f'dataset/test_preprocessed_div5k_cap20_meanAdj{meanAdjustment}_stdAdj{stdAdjustment}_max{test_df.INCWAGE_CPIU_2010.max()}_min{test_df.INCWAGE_CPIU_2010.min()}.csv', index=False)